In [2]:
import pandas as pd
import matplotlib.pyplot as plt
from ydata_profiling import ProfileReport

# import matplotlib
# matplotlib.use('WebAgg')


In [3]:
input_path = r'..\data\freight_data.csv'

raw_data = pd.read_csv(input_path, sep=',', parse_dates=['Date'])
raw_data.columns = [col.lower() for col in raw_data.columns]
raw_data.rename(
    mapper={column: column.lower() for column in raw_data.columns},
)
raw_data.head()

,carrier,trackingnumber,date,cost
0,UPS,1Z001,2021-01-01,40
1,UPS,1Z001,2021-01-01,40
2,UPS,1Z001,2021-02-01,70
3,UPS,1Z001,2021-02-01,20
4,UPS,1Z001,2021-03-01,40


In [4]:
raw_data.shape

(97, 4)

In [5]:
raw_data.drop_duplicates(inplace=True)
raw_data.shape

(95, 4)

In [6]:
raw_data.head()

,carrier,trackingnumber,date,cost
0,UPS,1Z001,2021-01-01,40
2,UPS,1Z001,2021-02-01,70
3,UPS,1Z001,2021-02-01,20
4,UPS,1Z001,2021-03-01,40
5,UPS,1Z002,2021-04-01,40


In [7]:
raw_data.describe()

,date,cost
count,95,95.000000
mean,2022-03-04 01:00:37.894736896,64.263158
min,2021-01-01 00:00:00,9.000000
25%,2021-10-07 00:00:00,40.000000
50%,2022-01-01 00:00:00,60.000000
75%,2022-10-07 00:00:00,88.000000
max,2022-12-05 00:00:00,222.000000
std,NaN,40.860481


In [8]:
profile = ProfileReport(raw_data, minimal=False, title='Raw Data Exploratory Analysis')

In [9]:
profile.to_widgets()

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render widgets:   0%|          | 0/1 [00:00<?, ?it/s]

In [10]:
# plt.scatter(x='date', y='cost', data=raw_data)
# plt.legend()
# plt.show()

In [ ]:
import matplotlib.dates as mdates
import matplotlib
matplotlib.use('TkAgg')

plt.hist(raw_data.date, bins=45)
plt.title('Distrubution of dates in the dataset')
plt.xticks(rotation=45, ha='right')
plt.show()